In [11]:
import numpy as np
from numpy.random import randint
from causallearn.utils.cit import CIT
from causallearn.search.ConstraintBased.FCI import fci
from causallearn.utils.GraphUtils import GraphUtils
from copy import  deepcopy

Apple_size = {
    1: "large size",
    -1: "small size"
}

Apple_smell = {
    1: "strong aroma",
    -1: "musty or rotten"
}

Apple_taste = {
    1: "more sweet than sour",
    -1: "more sour than sweet"
}

Apple_color = {
    1: "vibrant and uniformly colored",
    -1: "unevenly colored"
}

Apple_nutrition = {
    1: "a high content of essential nutrients, including dietary fiber, vitamin C, etc.",
    -1: "a relatively low content of essential nutrients"
}

Apple_juiciness = {
    1: "abundant and refreshing moisture",
    -1: "dry and lacking Moisture"
}

Apple_recmd = {
    1: "has significant market potential and deserves wider recognition",
    -1: "might not bring the expected returns and could even lead to losses",

}

map_0_1 = lambda x: 1 if x == 1 else -1


def get_fea():

    size = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1

    nutrition = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1  


    smell = map_0_1(np.random.uniform() < 0.5) * (np.random.uniform() < 0.9) * 1

    taste = map_0_1(np.random.uniform() < 0.5 ) * (np.random.uniform() < 0.9) * 1

    juiciness = taste * map_0_1(np.random.uniform() < 0.7 ) * (np.random.uniform() < 0.9) * 1
    
    score =  taste +  size + smell +  map_0_1(np.random.uniform() < 0.5 ) * (np.random.uniform() < 0.9) * 1

    recmd = 0
    if np.abs(score) > 1:
        recmd = np.sign(score )
    else:
        recmd = np.sign(nutrition + score)
    recmd *=  (np.random.uniform() < 0.9) * 1


    fea = np.array([score, nutrition, size, smell, taste, juiciness, recmd]) 
    return fea

names = ['score', 'nutrition', 'size', 'smell', 'taste', 'juiciness', 'recmd']

In [13]:
values = np.stack([get_fea() for _ in range(200)])

meta_ci_test = CIT(values, 'kci')
print(meta_ci_test(0, 6, [2, 3, 4]), meta_ci_test(0, 6, [1, 2, 3, 4, 5]))
print(meta_ci_test(0, 1, [2, 3, 4, 6]), meta_ci_test(0, 1, [ 2, 3, 4, 5, 6]))

5.048539364338467e-11 8.379888338794217e-09
5.430492722169333e-10 7.09341756444104e-08


In [21]:


def check(g):
    cond = True 
    cond &= g.graph[0,2] == 1
    cond &= g.graph[2,0] != 1

    cond &= g.graph[0,3] == 1
    cond &= g.graph[3,0] != 1

    cond &= g.graph[0,4] == 1
    cond &= g.graph[4,0] != 1

    cond &= g.graph[4,5] != 0

    cond &= g.graph[6,0] == 1

    cond &= g.graph[6,1] == 1
    return cond

t = 0
while t<500:
    t += 1

    values = np.stack([get_fea() for _ in range(200)])

    meta_ci_test = CIT(values, 'kci')
    
    result = max(meta_ci_test(0, 6, [2, 3, 4]), meta_ci_test(0, 6, [1, 2, 3, 4, 5])) < 1e-3
    if not result:
        continue
    
    result = max(meta_ci_test(0, 1, [2, 3, 4, 6]), meta_ci_test(0, 1, [ 2, 3, 4, 5, 6])) < 1e-3
    if not result:
        continue
    
    # some ind condition
    result = min( meta_ci_test(2, 6, [0]), meta_ci_test(3, 6, [0]), meta_ci_test(4, 6, [0]), meta_ci_test(5, 6, [0]) ) >0.05
    if not result:
        continue
    
    # Causal Discovery (FCI)
    for ind_method in [ 'gsq', 'kci']: # 'fisherz', 'chisq', 'gsq'
        g, _ = fci(values[:,:7], alpha = 0.01, independence_test_method=ind_method, verbose=False)
        result = check(g)
        if not result:
            break
        g, _ = fci(values[:,:7], alpha = 0.05, independence_test_method=ind_method, verbose=False)
        result = check(g)
        if not result:
            break
    
    if result:
        print(t)
        break
# visualization
pdy = GraphUtils.to_pydot(g, labels=names[:7])
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 999.46it/s]


X1 --> X7


Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 876.08it/s] 


X1 --> X7


Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 269.27it/s]


X1 --> X7


Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 184.27it/s]


X1 --> X7
1
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=nutrition];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [22]:
g, _ = fci(values[:,:7], alpha = 0.01, independence_test_method='gsq', verbose=False)
# visualization
pdy = GraphUtils.to_pydot(g, labels=names[:7])
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 939.67it/s]

X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=nutrition];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [23]:
g, _ = fci(values[:,:7], alpha = 0.05, independence_test_method='gsq', verbose=False)
# visualization
pdy = GraphUtils.to_pydot(g, labels=names[:7])
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 616.56it/s]

X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=nutrition];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [24]:
g, _ = fci(values[:,:7], alpha = 0.05, independence_test_method='kci', verbose=False)
# visualization
pdy = GraphUtils.to_pydot(g, labels=names[:7])
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 254.42it/s]


X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=nutrition];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [25]:
g, _ = fci(values[:,:7], alpha = 0.01, independence_test_method='kci', verbose=False)
# visualization
pdy = GraphUtils.to_pydot(g, labels=names[:7])
print(pdy.to_string())

Depth=0, working on node 6: 100%|██████████| 7/7 [00:00<00:00, 218.73it/s]


X1 --> X7
digraph  {
dpi=200;
fontsize=18;
0 [label=X1];
0 [label=score];
1 [label=X2];
1 [label=nutrition];
2 [label=X3];
2 [label=size];
3 [label=X4];
3 [label=smell];
4 [label=X5];
4 [label=taste];
5 [label=X6];
5 [label=juiciness];
6 [label=X7];
6 [label=recmd];
2 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
3 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 0  [arrowhead=normal, arrowtail=odot, dir=both];
0 -> 6  [arrowhead=normal, arrowtail=none, dir=both];
1 -> 6  [arrowhead=normal, arrowtail=odot, dir=both];
4 -> 5  [arrowhead=odot, arrowtail=odot, dir=both];
}



In [26]:
good_value = deepcopy(values)

In [28]:
import pandas as pd

In [29]:
pd.DataFrame(values, columns=names)

,score,nutrition,size,smell,taste,juiciness,recmd
0,-3,-1,-1,0,-1,-1,-1
1,0,1,-1,1,-1,1,1
2,3,-1,1,1,0,0,1
3,2,-1,1,-1,1,1,1
4,1,1,1,-1,1,1,1
...,...,...,...,...,...,...,...
195,2,1,-1,1,1,1,1
196,-4,-1,-1,-1,-1,1,-1
197,0,1,1,-1,-1,-1,1
198,-1,-1,0,-1,1,-1,-1


In [31]:
def get_prompt(score, nutrition, size, smell, taste, juiciness, recmd):
  # conifig
  # You are an excellent story writter, full of imagination. You will finish your next masterpiece based on the given materials.
  introduction = 'You are a picky gastronome on apples with your own standard.'
  Reasons_adjustment = 'write one concrete and interesting reseaon.' # 'write one concrete reseaon.'
  Description_adjustment = 'with creative description.' #'with creative, vivid, diverse and detailed words. Avoid cliche.' # 'with creative and detailed words.'


  apple_feature_list = []

  for fea_v, fea_dict in zip(
    [nutrition, size, smell, taste, juiciness, recmd],
    [Apple_nutrition, Apple_size, Apple_smell, Apple_taste, Apple_juiciness, Apple_recmd]
    ):
    if fea_v in fea_dict.keys():
      apple_feature_list.append('- '+fea_dict[fea_v])
  apple_feature = '\n'.join(apple_feature_list)

  authors = ['Shakespeare', 'a musician', 'master Yoda']
  prompt = f'''

You are a picky gastronome on apples. You are ready to **evaluate apples and write reviews**. Your writing should be **clear, solid and convincing to suppliers and customers**.

##  Task

Please write a short reveiw about the evaluation results for a given apple.

Evaluation Results:
{apple_feature}


Additional Requirement:
- Combine those evaluation results into more detailed reasoning.
- Single paragraph; No quotation marks.
- Modern English
- Begin this part by a mark [Review Begin].
- no more than 60 words.

  '''
  return prompt

In [32]:
print(get_prompt(*values[1]))



You are a picky gastronome on apples. You are ready to **evaluate apples and write reviews**. Your writing should be **clear, solid and convincing to suppliers and customers**.

##  Task

Please write a short reveiw about the evaluation results for a given apple.

Evaluation Results:
- a high content of essential nutrients, including dietary fiber, vitamin C, etc.
- small size
- strong aroma
- more sour than sweet
- abundant and refreshing moisture
- has significant market potential and deserves wider recognition


Additional Requirement:
- Combine those evaluation results into more detailed reasoning.
- Single paragraph; No quotation marks.
- Modern English
- Begin this part by a mark [Review Begin].
- no more than 60 words.

  


In [ ]:
import sys

from openai import OpenAI
from scripts.neuropathic.utils import Logger
from scripts.neuropathic.openai_utils import LLM_openai
import time

logger = Logger(
    root = '<specify the address>', 
    exp_name = 'build_AG7L1_0512',
    
)
llm = LLM_openai(logger, api_key="<specify the api key>")


In [40]:
reviews = []
sample_size = 200


for idx in range(sample_size):
    prompt =get_prompt(*values[idx])

    response = llm.chat(
        model="gpt-4-1106-preview",
        content= prompt,
        system_instruct = "You are a helpful assistant."
        )
    
    if '[Review Begin]' in response:
        review = response.split('[Review Begin]')[-1]
    elif 'Part 2, Review:' in response:
        review = response.split('Part 2, Review:')[-1]
    elif 'Part 2, Review.' in response:
        review = response.split('Part 2, Review.')[-1]
    elif 'Part 2, Review\n' in response:
        review = response.split('Part 2, Review\n')[-1]
    else:
        print(f'{idx}-th sample need help')
        review = response


    reviews.append(review)
    time.sleep(1)

chatcmpl-9OLR1BUeoOYvNWEiVck4N50n84z5y
chatcmpl-9OLR7bdDa5KYyfGvuIiQgaFdWe6dS
chatcmpl-9OLREcfVCMFUlvlWKErEPK6KPP6NK
chatcmpl-9OLRIOewiAiZmIsppaEZQAju3eOdG
chatcmpl-9OLRNG4CZ5rS8GfNgFgTIPr7ZZBfN
chatcmpl-9OLRUoCyf6etYeHhIMJQ6yAAlXIc0
chatcmpl-9OLRbli0HCQmJ7Lf7BYkeMBynqJ7r
chatcmpl-9OLRiLyD4BZtjnnNbK6maqvVUmHi4
chatcmpl-9OLRpXgzbrqsGACf2sg6codyRQuJ5
chatcmpl-9OLRuQl2hwQolUOhzGWXQivrEolAj
chatcmpl-9OLS0LeAwoA83JwmKwYjYrONPmGVS
chatcmpl-9OLS53EW5wm3dbIJdthzDqK3k7PWj
chatcmpl-9OLSA6KxaW0HvF7BJdEOuLy1SNYbx
chatcmpl-9OLSGgZXu8P2hlxrxJ4A98QCh7Akn
chatcmpl-9OLSLfvcN1gCM7FAX1dUB5jLbe94W
chatcmpl-9OLSQ9v7dEBd0SAsMuZv2zu5ZV3vf
chatcmpl-9OLSUrfQwEMXYuFMhpJesht143CuB
chatcmpl-9OLSZsg30DTgdPqlq3sMTOJ8PzFtS
chatcmpl-9OLSduW6hYvgci2yEi6EsLa9NEiMl
chatcmpl-9OLShODnGzfNR5wR2zs9cz0CrvrEh
chatcmpl-9OLSl4Bz20kdxmHTa7ADNUQtRdASD
chatcmpl-9OLSqGmBxYk2KTjJElGmJ1n5jF1kV
chatcmpl-9OLSvETZdoEj9Vg1cth6XAyaHCqSW
chatcmpl-9OLT3qc0BU1m9XECFbUza7j6iUvAS
chatcmpl-9OLT98cJffqSMZoyAhhgHnmhzcdzw
chatcmpl-9OLTED1PgBcHntht

In [41]:
import pandas as pd

pd_data = pd.DataFrame(values, columns=names)
pd_data['Review'] = reviews
pd_data.head()

,score,nutrition,size,smell,taste,juiciness,recmd,Review
0,-3,-1,-1,0,-1,-1,-1,The assessed apple falls short in offering th...
1,0,1,-1,1,-1,1,1,This apple's profile—rich in essential nutrie...
2,3,-1,1,1,0,0,1,Despite the comparatively modest nutrient pro...
3,2,-1,1,-1,1,1,1,Despite its large size and appealing sweetnes...
4,1,1,1,-1,1,1,1,Despite its impressive profile packed with di...


In [42]:
pd_data.isna().any()

score        False
nutrition    False
size         False
smell        False
taste        False
juiciness    False
recmd        False
Review       False
dtype: bool

In [43]:
pd_data.to_excel("Apple_Gastronome_AG7_v20240513_tobe_checked.xlsx", index=False) 

In [44]:


score_set = pd_data['score'].unique()
score_set = score_set[np.argsort(score_set)]

sample_size = pd_data.shape[0]

sorted_data = '# Data\n'
for v in score_set:
    sorted_data +=  f'\n## Score: {v}\n'
    this_indeces = np.arange(sample_size)[pd_data['score'].values==v]
    for this_idx in this_indeces:
        this_review = pd_data.values[this_idx,-1].replace("\n", " ")
        
        sorted_data +=  f'\n{this_review}\n'

print(sorted_data)


# Data

## Score: -4

 This apple variety, regrettably undersized and with a tendency towards mustiness, disappoints with its imbalance, leaning more into sourness than desired sweetness. The unexpected dryness further diminishes its appeal, suggesting potential struggle in the market and significant risk for unprofitable returns, making it a risky choice for suppliers and customers alike.

 This apple variety, regrettably, underperforms with its diminutive stature and unsavory mustiness that veers towards a rotten experience. Its flavor profile skews excessively sour over sweet, overshadowing its redeeming quality of a juicy, thirst-quenching bite. As such, it is unlikely to meet financial forecasts, posing a significant risk of economic shortfall for vendors and disappointment for consumers.

 This apple variety, with its notably low nutrient profile and diminutive stature, falls short in meeting desired quality marks. A pervasive mustiness indicates potential spoilage, detracting fr